<img src='./images/Rectangle.svg' width=400 height=400/>


**<u>Mo</u>**lecular **<u>S</u>**imulation **<u>De</u>**sign **<u>F</u>**ramework, or MoSDeF, is a robust Python-based, open-source software framework designed to faciliate the initialization, atom-typing, and screening of soft matter systems for molecular simulation. The project was initially developed at Vanderbilt University, in collaboration with software engineers from the Vanderbilt Institute for Software Integrated Systems (ISIS). The project has now expanded into a multi-university collaboration with Vanderbilt University as the lead institution. 


The MoSDeF software suite comprises three main libraries: `mbuild`, `foyer`, and `GMSO`.  Each library targets a different step of system initialization. Specifically, the `mbuild` library can be used to systematically construct any molecular system, including atomistic and coarse-grained molecules. `Foyer` can be used to assign interaction parameters to all particles, bonds, angles, and dihedrals in the created system. Finally, the `GMSO` library, currently under development, will serve as the main data structure to store all the information of the system, including the details of the system (particles and their positions) along with interaction types and parameters of that system. 


The MoSDeF tools allow users to build complicated systems in a manner that is scriptable and automatable.   This enables molecular simulation studies to be more transparent and reproducible, as these scripts can be easily shared on software hosting services, such as GitHub.  Automation also allows for rapid system initialization for large-scale screening studies, a rapidly growing subset of the molecular simulation field.

## Notebook Overview

In this notebook, we will demonstrate how MoSDeF can be used in a molecular simulation workflow. Along the way we will highlight important features of each package. In context of a short demonstration, we will run a simple simulation (equilibrium simulation of a box of small molecules). At the end of the notebook, examples of more complicated workflows/tutorials, which were inspired by previously published work, are included.

For demonstration purposes, we will be using [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/) to show off various functionality of MoSDeF.  Note that typical system initialization workflows using MoSDeF are entirely written in Python and do not involve the use of Widgets. Once any of the widgets are activated, it will also print the script of what is happening under the hood.

For reproducible molecular simulations, using graphical user interfaces (GUIs) are discouraged, as these tools usually obfuscate the specific settings and parameters needed to replicate the work by an external user. Although, in demonstrations and presentations, GUIs can be quite valuable as a visual aid.

### mBuild

Check out `mbuild` here:
* https://github.com/mosdef-hub/mbuild
* https://mbuild.mosdef.org

`mBuild` is the molecule building library within MoSDeF. This library includes features that allow users to create complex molecular systems starting from the particle level. The core data structure in `mbuild` is the `Compound` class, which can be used to represent anything from a single particle to a complete system with ten of thoudsands of particles.  
<img src='images/pmpc.png' width=800 height=800/>
With the hierarchial (nestable), component-based design, users can switch out components programmatically to create new systems. The code to create these systems can also be organized into separate Python plugins, which we call `mbuild Recipes`.

`mBuild` allows the initialization of `Compound` objects in several different ways.  Here we demonstrate the functionality of loading a `Compound` from the 2D representation of a molecular structure known as SMILES (Simplified Molecular Input Line Entry System) string.

In [ ]:
import mbuild as mb
# Loading a compound from a SMILES string
compound = mb.load('C1CCCCC1', smiles=True)
compound.visualize()

Initialization of a molecule from a SMILES string is a handy tool to quickly create small molecules, since they can be easily found in a public database, such as https://pubchem.ncbi.nlm.nih.gov/ 


In the demonstration below, you can try to enter a SMILES string, and the `mbuild` library will read it in, turn it into a `mbuild.Compound` and visualize it with `py3dmol`! 


If you don't have one in mind, try:
- Caffeine: `CN1C=NC2=C1C(=O)N(C(=O)N2C)C`
- Nicotine: `CN1CCCC1C2=CN=CC=C2`
- Cholesterol: `CC(C)CCCC(C)C1CCC2C1(CCC3C2CC=C4C3(CCC(C4)O)C)C`

In [ ]:
import src.widget_events as widgets
import ipywidgets as iwidgets

display(widgets.smiles_box)
display(widgets.out_smiles)

`mbuild.load()` can easily load in a wide range of common molecule file formats, such as `pdb`, `mol2`, `xyz`, etc.

In this example workflow, we will demonstrate the process of loading small molecules from `mol2` files located in this repo to build a box of molecules, where users can specify the size of the box and the number of particles in the box. 

In [ ]:
display(widgets.compound_dropdown)
display(widgets.out_mol)

In [ ]:
display(iwidgets.VBox([widgets.box_slider, widgets.n_slider]))
display(widgets.out_box)

A detailed series of tutorials showcasing features and use-cases of `mbuild` can be found at https://github.com/mosdef-hub/mosdef_tutorials


Due to restriction of forcefield parameters availability, the example workflow will continue with the box of compound selected from the drop down list.

### Foyer
Check out `foyer` here:
* https://github.com/mosdef-hub/foyer/
* https://foyer.mosdef.org

`Foyer` is the atom-typing and parameterization library within MoSDeF. In molecular simulation, the bonded and non-bonded parameters that define the interactions between particles are called a force field.  These interaction parameters are derived by fitting to a specific physical property, such as density.


In `foyer`, force field information is contained within an XML file, that can be loaded in as a `Forcefield` object.  By calling `Forcefield.apply()` on an `mbuild.Compound`, if matches are found, the force field parameters are automatically applied to the compound, returning a fully parametrized system.


Below we will demonstrate the process of initializing a `Forcefield` object with foyer, and applying the force field parameters to the `mbuild.Compound` initialized above.  We will use the General Amber Force Field (GAFF) using the `GAFF-foyer` plugin, and we will assign partial charges to each atom by using the `antefoyer` plugin.

These are two great examples of the extensibility of the MoSDeF tools and its robust setuptools-inspired plugin system. These plugins can be developed independently of `mbuild` or `foyer` and when installed, are dynamically loaded by their respective package.

In [ ]:
import foyer
import antefoyer

gaff_ff = foyer.forcefields.load_GAFF()

if widgets.BOX_OF_COMPOUNDS:
    typed_compound = gaff_ff.apply(widgets.BOX_OF_COMPOUNDS,
                              assert_dihedral_params=False)
else:
    print('Please pick a molecule from the dropdown above.')

We have written a small function to help apply the partial charges with `antefoyer` and `antechamber`.  The charge assignment method we are using is `AM1BCC`, and we are setting the net charge to `0.0`.

In [ ]:
def apply_charges(box_structure, single_compound, n_atoms, ff):
    single_mol_struct_charge = antefoyer.ante_charges(
            single_compound, 'bcc', net_charge=0.00, multiplicity=1)
    
    for index, atom in enumerate(box_structure.atoms):
        atom.charge = single_mol_struct_charge.atoms[index%n_atoms].charge
    return box_structure

In [ ]:
charge_structure = apply_charges(box_structure=typed_compound,
                                 single_compound=widgets.COMPOUND,
                                 n_atoms=widgets.COMPOUND.n_particles,
                                 ff=gaff_ff)

### GMSO (under development) 
Check out `GMSO` here:
- https://github.com/mosdef-hub/gmso
- https://gmso.mosdef.org

`GMSO` is designed to be a general and flexible representation of chemical topologies for molecular system. The package is currently under development, but it is set to become the default backend data structure for `foyer` to store all the information of a typed molecular system.

The main attribute that sets `GMSO` apart from similar libraries is its generality. By assuming as little as possible about the chemical system, model, or engine, `GMSO` can support a variety of systems with different type of force fields and interaction expressions. 

In [ ]:
import gmso
import gmso.external
import gmso.formats
import warnings
warnings.filterwarnings("ignore")

topology = gmso.external.from_parmed(charge_structure)
topology.name = widgets.COMPOUND.name

`GMSO` expects explicit definitions when defining forcefield parameters. Hence, the library utilizes [sympy](https://sympy.org), specifically the `sympy.expression` module to store the expression for all functional forms used to define molecular interactions (AtomType, BondType, etc.), and the [unyt library](https://unyt.readthedocs.io) to keep track of all corresponding parameters' units. An example of an AtomType's expression and its parameters are shown below.

This example defines the particle-particle interactions between any two pairs of particles using the functional form of the [Lennard-Jones potential](https://en.wikipedia.org/wiki/Lennard-Jones_potential), where the parameters `sigma` and `epsilon` are defined for each particle in the system.

In [ ]:
display(topology.sites[0].atom_type.expression)
display(topology.sites[0].atom_type.parameters)

Last but not least, the most important task of `GMSO`, or any chemical topolgy backend, is to save out accurate molecular topology files that can be read and executed by various simulation engines. Since generality is one of the main focuses of `GMSO`, we plan to support writing out to as many types of molecular file formats as possible. Currently, we already have writers for a few popular simulation codes such as LAMMPS and GROMACS.

Using `sympy` to define the various functional forms that could possibly be supported by various engines grants `GMSO` the ability to write to any simulation engine that might support said functional form, a feature that is lacking in most/all other chemical topology packages.

In [ ]:
gmso.formats.write_top(topology, "simulation/topol.top", 
                       top_vars={"fudgeLJ": 0.5, "fudgeQQ": 0.8, "comb-rule": "geometric"})
gmso.formats.write_gro(topology, "simulation/conf.gro")

Time to run the simulation! We are using GROMACS to simulate a short equilibrium simulation of small molecules in a box.

In [ ]:
from run_simulation import run_energy_minimization, run_nvt
run_energy_minimization()
run_nvt()

The result can be analyzed by other available analysis library such as [the `panedr` package](https://github.com/jbarnoud/panedr) which converts the energy file output by GROMACS into a `pandas` `DataFrame`.

In [ ]:
import panedr
import matplotlib.pyplot as plt
sim_data = panedr.edr_to_df("simulation/ener.edr")

plt.figure(dpi=350, figsize=(5, 5))
plt.plot(sim_data["Time"], sim_data["Temperature"])

We can visualize the movement of the molecules during the simualtion using [the `nglview` package](https://github.com/arose/nglview).

In [ ]:
from run_simulation import visualize_trajectory

visualize_trajectory()

The script to run this example workflow can be found in `workflow.py`, which is in the same directory as this notebook. Users can execute the script with the command `python workflow.py`, and follow the instructions to provide:
- Path to the molecule file
- Dimension of the box 
- Number of molecules in the box

### More links

- The above example only present a fraction of MoSDeF's capabilities. More information of about this software suite can be found at:
    - https://mosdef.org
    - https://github.com/mosdef-hub
- More example of MoSDeF workflows with more interesting systems can be found at:
    - https://github.com/mosdef-hub/mosdef-workflows
    - https://github.com/daico007/TRUE-nanotribology
    - https://github.com/uppittu11/true_lipids
    - https://github.com/rmatsum836/true_graphene